# Web Scrapping 

In [7]:
# import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

# part 1
this part focuses on extracting links to the statistics for each of the teams in the landing page. the links are contained
in the 

In [27]:
# site url 
url = "https://fbref.com/en/comps/9/Premier-League-Stats"

# request the data
response = requests.get(url, timeout=5)

# initialize BeautifulSoup with HTML content(or text) and parser
soup = BeautifulSoup(response.text, 'html.parser')
soup

<!DOCTYPE html>
<html lang="en-US"><head><title>Just a moment...</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="noindex,nofollow" name="robots"/><meta content="width=device-width,initial-scale=1" name="viewport"/><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;max-width:60rem;padding-left:1.5rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{font-size:1.5rem;font-weight:500;line-height:2.25rem}@media (width <= 720px){.h2{font-size:1.25rem;line-height:1.5rem}}#challenge-error-text{background-image:url();background-repeat:no-repeat;backgro

In [ ]:
# 